In [13]:
# from project_forntend import call_llm
import sys
import os
sys.path.insert(0,"/home/sangram/python_pratice/llm/test_llm")
print(sys.path)
from  project_forntend.LLM import call_llm
llm=call_llm()
print(os.getcwd())


['/home/sangram/python_pratice/llm/test_llm', '/usr/lib/python311.zip', '/usr/lib/python3.11', '/usr/lib/python3.11/lib-dynload', '', '/home/sangram/python_pratice/llm/llm_run/lib/python3.11/site-packages', '/home/sangram/python_pratice/llm/llm_run/lib/python3.11/site-packages/onnxruntime/transformers', '/home/sangram/python_pratice/llm/llm_run/lib/python3.11/site-packages/onnxruntime/transformers/../tools']
call from project_forntend
API key found and looks good so far!
/home/sangram/python_pratice/llm/test_llm/my_resume_project


In [1]:
from langchain.document_loaders import PyMuPDFLoader,UnstructuredPDFLoader
resume='/home/sangram/python_pratice/llm/test_llm/my_resume_project/Resume_sangram_SR_Manager_latest2025.pdf'
loader_resume=PyMuPDFLoader(resume)
loader_resume_un=UnstructuredPDFLoader(resume,partition_via_api = True)
pages=loader_resume.load()
pages_un=loader_resume_un.load()

ImportError: cannot import name 'PSSyntaxError' from 'pdfminer.pdfparser' (/home/sangram/python_pratice/llm/llm_run/lib/python3.11/site-packages/pdfminer/pdfparser.py)

In [2]:
pages

[Document(metadata={'producer': 'LibreOffice 6.4', 'creator': 'Writer', 'creationdate': '2025-04-26T17:30:46+05:30', 'source': '/home/sangram/python_pratice/llm/test_llm/my_resume_project/Resume_sangram_SR_Manager_latest2025.pdf', 'file_path': '/home/sangram/python_pratice/llm/test_llm/my_resume_project/Resume_sangram_SR_Manager_latest2025.pdf', 'total_pages': 4, 'format': 'PDF 1.5', 'title': '', 'author': '', 'subject': '', 'keywords': '', 'moddate': '', 'trapped': '', 'modDate': '', 'creationDate': "D:20250426173046+05'30'", 'page': 0}, page_content='Sangram Singh,\nMobile -09958907649                                                       \nEmail: sangre53@gmail.com, \nsangram_singh23@yahoo.com, \nAddress: - Ameerpet, Hyderabad, Telangana\nExecutive Summary:\nDynamic and adaptable leader with 15+ years of experience across roles like Site \nrelibity Engineering ( SRE ), Operations and Production Support, and 1.9+  years in\nKPI-driven people management Leadership role. Known for quic

In [5]:
from langchain.text_splitter import CharacterTextSplitter
text_split=CharacterTextSplitter(
      separator="\n",
      chunk_size=1000,
      chunk_overlap=150,
      length_function=len
)
doc=text_split.split_documents(pages)

In [8]:
doc
len(doc),len(pages)

(10, 4)

remove Whitespace

In [9]:
def remove_ws(d):
      text=d.page_content.replace('\n','')
      d.page_content=text
      return d
doc=[remove_ws(d) for d in doc]

In [10]:
doc

[Document(metadata={'producer': 'LibreOffice 6.4', 'creator': 'Writer', 'creationdate': '2025-04-26T17:30:46+05:30', 'source': '/home/sangram/python_pratice/llm/test_llm/my_resume_project/Resume_sangram_SR_Manager_latest2025.pdf', 'file_path': '/home/sangram/python_pratice/llm/test_llm/my_resume_project/Resume_sangram_SR_Manager_latest2025.pdf', 'total_pages': 4, 'format': 'PDF 1.5', 'title': '', 'author': '', 'subject': '', 'keywords': '', 'moddate': '', 'trapped': '', 'modDate': '', 'creationDate': "D:20250426173046+05'30'", 'page': 0}, page_content='Sangram Singh,Mobile -09958907649                                                       Email: sangre53@gmail.com, sangram_singh23@yahoo.com, Address: - Ameerpet, Hyderabad, TelanganaExecutive Summary:Dynamic and adaptable leader with 15+ years of experience across roles like Site relibity Engineering ( SRE ), Operations and Production Support, and 1.9+  years inKPI-driven people management Leadership role. Known for quickly learning new

In [14]:
from langchain_openai.embeddings import OpenAIEmbeddings
import sys
import os
sys.path.insert(0,"/home/sangram/python_pratice/llm/test_llm")
print(sys.path)
from  project_forntend.LLM import import_llm_key 
embeddings=OpenAIEmbeddings(api_key=import_llm_key())

['/home/sangram/python_pratice/llm/test_llm', '/home/sangram/python_pratice/llm/test_llm', '/usr/lib/python311.zip', '/usr/lib/python3.11', '/usr/lib/python3.11/lib-dynload', '', '/home/sangram/python_pratice/llm/llm_run/lib/python3.11/site-packages', '/home/sangram/python_pratice/llm/llm_run/lib/python3.11/site-packages/onnxruntime/transformers', '/home/sangram/python_pratice/llm/llm_run/lib/python3.11/site-packages/onnxruntime/transformers/../tools']
API key found and looks good so far!


In [15]:
import faiss
from langchain_community.vectorstores import FAISS
db=FAISS.from_documents(doc,embeddings)

In [18]:
query="list the heading of every section from resume"
answer=db.similarity_search(query)
retriever=db.as_retriever(search_kwargs={'k':3})

In [22]:
from langchain_core.output_parsers import StrOutputParser
from langchain_core.prompts import ChatPromptTemplate
from langchain_core.runnables import RunnablePassthrough


In [23]:
from langchain.prompts import PromptTemplate


In [26]:
template="""
Persona:
You are an expert in writing resumes for people who are looking for jobs. You guide them on the best format to showcase their skills and experience. You take information from their current resume and rewrite it into a new, improved format using the same content.
context: {context}
query: {query}
"""

In [28]:
prompt_template=PromptTemplate.from_template(template)

In [35]:
chain=(
      {"context": retriever,
       "query": RunnablePassthrough()}|prompt_template|llm|StrOutputParser()
)

In [36]:
chain.invoke('List the main heading of candiate resume')

"The main headings of the candidate's resume are as follows:\n\n1. **Personal Information**\n2. **Executive Summary**\n3. **Technical Skills**\n4. **Professional Experience** (implied from the content)\n5. **Academic and Professional Qualification**\n6. **Personal Profile**\n7. **Declaration**"